# Скрейпинг и парсинг веб-страниц

**Скрейпинг** (от англ. web scraping) - технологии получения данных путем извлечения их со страниц веб-ресурсов.

**Парсинг** (от англ. parsing) - принятое в лингвистике и информатике определение синтаксического анализа (например, анализ синтаксиса языка разметки HTML или JSON-файлов). 

**BeautifulSoup** - python-библиотека для для синтаксического разбора файлов HTML/XML. В веб-разработке «суп из тегов» (tag soup) - это слово для синтаксически или структурно некорректного HTML, написанного для веб-страницы. 
Документация: https://www.crummy.com/software/BeautifulSoup/bs4/doc/

**Requests** - python-библиотека для отправки всех видов HTTP-запросов. 

**Pandas** — программная библиотека на языке Python для обработки и анализа данных. Название pandas образовано от термина panel data (панельные данные), применяемого в эконометрике для обозначения многомерных структурированных наборов данных, так и от фразы Python data analysis.

**Selenium Webdriver** - инструмент для автоматизации действий веб-браузера (программная библиотека, которая позволяет разрабатывать программы, управляющие поведением браузера). В большинстве случаев используется для тестирования Web-приложений, но и для скрейпинга тоже полезно. https://www.selenium.dev/documentation/en/webdriver/

# Один из возможных алгоритмов скрейпинга

* указать в коде адрес интересующего сайта: откуда вы хотите скачать данные?
для этого используется библиотека requests

* сохранить веб-страницу (html-код страницы)

* выбрать данные, которые нужно собрать (используется BeautifulSoup)

* записать данные в csv-файл. Это делается с помощью библиотеки csv. 

Если нужно соскрейпить несколько страниц - повторяем процесс для каждой из них.

# Пример № 1: используем библиотеки Requests и BeautifulSoup

In [ ]:
'''
установка библиотек

!pip install requests
!pip install bs4
!pip install pandas

SyntaxError: ignored

In [ ]:
#импортируем необходимые библиотеки
import requests
from bs4 import BeautifulSoup as bs
import csv
import pandas as pd

мы хотим получить данные из таблицы по коронавирусу из Википедии
https://ru.wikipedia.org/wiki/%D0%A0%D0%B0%D1%81%D0%BF%D1%80%D0%BE%D1%81%D1%82%D1%80%D0%B0%D0%BD%D0%B5%D0%BD%D0%B8%D0%B5_COVID-19_%D0%B2_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8

In [ ]:
#формируем запрос
url = 'https://ru.wikipedia.org/wiki/%D0%A0%D0%B0%D1%81%D0%BF%D1%80%D0%BE%D1%81%D1%82%D1%80%D0%B0%D0%BD%D0%B5%D0%BD%D0%B8%D0%B5_COVID-19_%D0%B2_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8'
page = requests.get(url)

In [ ]:
#проверим, что все прошло хорошо (какой ответ пришел на наш запрос?)
page

<Response [200]>

Теперь, когда у нас есть страница, давайте проанализируем ее. 

Для синтаксического анализа (скрейпинга) хорошо подходит **BeautifulSoup (page.content, 'html.parser')**.

Сохраним в переменную 'soup' весь HTML-код страницы. HTML-код - это "дерево тегов", формирующее контент страницы.

In [ ]:
soup = bs(page.content, 'html.parser')
soup

KeyboardInterrupt: ignored

Теперь из этого "супа" мы хотим получить таблицу с данным. 

**Функция soup.find('table')** найдет первый в дереве тег table.
Если нам нужно найти не только первый элемент, а все элементы по определеннопу признаку, следует использовать функцию **soup.find_all**

В переменную table сохраним ту часть HTML-кода, которая формирует таблицу

In [ ]:
table = soup.find_all('table')
table
table[8] 

In [ ]:
#в таблице строки формируются тегом <tr>
#найдем все строки

rows = table[8].find_all('tr')
rows[10]

#какой объект мы получили?

In [ ]:
#теперь мы можем обращаться к каждой строке таблицы, как к элементу списка - по индексу

rows[15]

In [ ]:
#значение каждой ячейки таблицы в cтроке ограничевается тегом <td>

rows[15].find_all('td')

In [ ]:
#кроме того нам нужно название региона, оно прячется в теге <th> 
rows[15].find_all('th')

Итак, мы нашли нужные элементы таблицы. Теперь нам нужно собрать их все (по всем регионам)

Что нужно делать:

- создать пустой список, в который будем добавлять элементы для формирования таблицы;
- извлечь таблицу из супа, сохранить ее в переменной table
- с помощью цикла "перебрать" строки таблицы и извлечь из каждой нужные данные
- добавить выбранные данные в список

In [ ]:
#собираем код
#для каждой строки таблицы
covid_data = []
for row in table[8].find_all('tr')[4:]:
    region = row.find('th').text
    d1 = row.find_all('td')[0].text
    d2 = row.find_all('td')[0].text
    d3 = row.find_all('td')[0].text
    d4 = row.find_all('td')[0].text
    d5 = row.find_all('td')[0].text
    rowData = [region, d1, d2, d3, d4, d5]
    print(rowData)
    covid_data.append(rowData)

print(len(covid_data))

In [ ]:
frame = pd.DataFrame(covid_data)

In [ ]:
frame

In [ ]:
frame.to_csv("covid_data.csv", index=False)

Ура!!! Мы собрали данные из таблицы в Википедии. 

Осталось совсем немного - почистить наши строки от непечатаемых символов, объединить все в датафрейм и сохранить в csv.

# Пример №2: скрейпим сайт Фонда президенстких грантов с помощью Selenium


## Внимание! Selenium работает только из ос. Из колаба он не запустится.
В данном блокноте приведен пример работы с selenium.

Подробное руководство здесь: https://www.selenium.dev/documentation/en/webdriver/

In [ ]:
!pip install webdriver-manager

In [ ]:
#Если selenium не установлен, нужно его установить так: !pip install selenium
#Дальше вся работа в браузере Google Chrome
#wd = webdriver.Chrome(executable_path=r'C:\Program Files (x86)\Google\Chrome\Application\chromedriver.exe')

In [ ]:
#импортируем необходимые библиотеки и модули

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import os
from webdriver_manager.chrome import ChromeDriverManager 

# # Формируем запрос

В этом примере мы будем работать с сайтом Фонда президентских грантов: https://президентскиегранты.рф

На этот раз нам нужно соскрейпить не одну страницу сайта, а несколько.
Сделать это поможет цикл. Мы должны пройти циклом по всем url-адресам.

**Как это сделать?** - изучите url-ы нашего сайта. 
 - Отличается ли url первой страницы от последующих?
 - Не напоминает ли вам url запросы к API, которые мы делали раньше?
 
 В нашем случае все достаточно просто. Есть адрес первой страницы - без параметра page.
 Во всех остальных случаях в конце url-а добавляется параметр page = 'номер страницы'.
 
 Сколько циков пройдет в нашем коде?

In [ ]:
first_url = 'https://xn--80afcdbalict6afooklqi5o.xn--p1ai/public/application/cards?SearchString=&Statuses%5B0%5D.Selected=true&Statuses%5B0%5D.Name=%D0%BF%D0%BE%D0%B1%D0%B5%D0%B4%D0%B8%D1%82%D0%B5%D0%BB%D1%8C+%D0%BA%D0%BE%D0%BD%D0%BA%D1%83%D1%80%D1%81%D0%B0&Statuses%5B1%5D.Name=%D0%BD%D0%B0+%D0%BD%D0%B5%D0%B7%D0%B0%D0%B2%D0%B8%D1%81%D0%B8%D0%BC%D0%BE%D0%B9+%D1%8D%D0%BA%D1%81%D0%BF%D0%B5%D1%80%D1%82%D0%B8%D0%B7%D0%B5&Statuses%5B2%5D.Name=%D0%BF%D1%80%D0%BE%D0%B5%D0%BA%D1%82+%D0%BD%D0%B5+%D0%BF%D0%BE%D0%BB%D1%83%D1%87%D0%B8%D0%BB+%D0%BF%D0%BE%D0%B4%D0%B4%D0%B5%D1%80%D0%B6%D0%BA%D1%83&RegionId=&AreaCityId=&ContestDirectionTenantId=&IsNormalTermProjects=true&IsLongTermProjects=true&CompetitionId=63498846-9978-4c0b-95a3-beed6f81ff85&DateFrom=&DateTo=&Statuses%5B0%5D.Selected=false&Statuses%5B1%5D.Selected=false&Statuses%5B2%5D.Selected=false&IsNormalTermProjects=false&IsLongTermProjects=false'

other_url = 'https://xn--80afcdbalict6afooklqi5o.xn--p1ai/public/application/cards?SearchString=&Statuses%5B0%5D.Name=%D0%BF%D0%BE%D0%B1%D0%B5%D0%B4%D0%B8%D1%82%D0%B5%D0%BB%D1%8C+%D0%BA%D0%BE%D0%BD%D0%BA%D1%83%D1%80%D1%81%D0%B0&Statuses%5B1%5D.Name=%D0%BD%D0%B0+%D0%BD%D0%B5%D0%B7%D0%B0%D0%B2%D0%B8%D1%81%D0%B8%D0%BC%D0%BE%D0%B9+%D1%8D%D0%BA%D1%81%D0%BF%D0%B5%D1%80%D1%82%D0%B8%D0%B7%D0%B5&Statuses%5B2%5D.Name=%D0%BF%D1%80%D0%BE%D0%B5%D0%BA%D1%82+%D0%BD%D0%B5+%D0%BF%D0%BE%D0%BB%D1%83%D1%87%D0%B8%D0%BB+%D0%BF%D0%BE%D0%B4%D0%B4%D0%B5%D1%80%D0%B6%D0%BA%D1%83&RegionId=&ContestDirectionTenantId=&CompetitionId=91c698a4-c11d-4ad6-9148-03d58cd7c837&DateFrom=&DateTo=&Statuses%5B0%5D.Selected=false&Statuses%5B1%5D.Selected=false&Statuses%5B2%5D.Selected=false&page={}'

In [ ]:
#после запуска этого кода должно открыться новое окно с гугл поиском по запросу
#ЕГО НЕ НАДО ЗАКРЫВАТЬ!

  browser = webdriver.Chrome(ChromeDriverManager().install())
browser.set_window_size(1280, 1024)
browser.get(first_url)

IndentationError: ignored

# Изучаем html-код нашего сайта

вот так выглядит html-код нужного нам элемента. 
Мы знаем, что таких элементов по 20 шт. на страницу. 
Всего ??? страниц.
На последней странице только ??? элементов.
Один элемент - одна строка в нашем будущем датасете. 
Сколько всего он должен содержать строк?


Мы будем искать элементы по имени класса. Для этого сайта это подойдет.


## Но возможны и другие варианты 

- find_element by_id
- find_element by_name
- find_element by_xpath
- find_element by_link_text
- find_element by_partial_link_text
- find_element by_tag_name
- find_element by_class_name
- find_element by_css_selector

# RTFM!!!


In [ ]:
<div class="table__row cards-item-row">
        <div class="table__cell cards-item-cell">
            <div class="projects__title">Творческие мастерские для детей с инвалидностью</div>
            <div class="projects__title-sub">
                <div class="contest">первый конкурс 2017</div>
                <div class="direction">Поддержка семьи, материнства, отцовства  и детства</div>
            </div>
            <div class="js-p-clone-to"></div>
        </div>
        <div class="table__cell js-p-clone-from cards-item-cell">
            
                <div class="projects__price">
                    1 700 000,00&nbsp;<span class="rubl">₽</span>
                </div>
                    <div class="projects__fond">
                        <div>Перечислено Фондом</div>
                        <div class="projects__price projects__price--fond">
                                <div class="tooltip tooltip--round">
                                    <div class="tooltip__btn">
                                        <svg><use xlink:href="#info"></use></svg>
                                    </div>
                                    <div class="tooltip__box">
                                        Получено грантополучателем <br> на&nbsp;1&nbsp;число текущего месяца<br> (грант предоставляется платежами в&nbsp;соответствии с&nbsp;графиком отчетности и&nbsp;платежей по&nbsp;гранту)
                                    </div>
                                </div>
                            <span class="projects__str-no-wrap">1 700 000,00&nbsp;<span class="rubl">₽</span></span>

                        </div>
                    </div>

            <div class="projects__descr">
                <div>Кировская область</div>
                <div class="projects__type">
                        <span class="c--green">победитель конкурса</span>
                        <span class="projects__type-rate">
                            <span><svg><use xlink:href="#star-yellow"></use></svg></span><span>97,5</span>
                        </span>
                </div>
                <span>
                        <span class="projects__str-no-wrap">Заявка&nbsp;17-1-005875 от 31.05.2017</span>

                </span>
            </div>
        </div>
    </div>

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 26)

**Итак, мы хотим выгрузить следующие данные по каждой заявке на грант:**

1. Название проекта;
2. Конкурс;
3. Сумма гранта;
4. Направление гранта;
5. Регион участника;
6. Статус участника;
7. № заявки

In [ ]:
#получаем нужную информацию по имени класса
#если мы хотим получить не только один элемент, а все элементы соответствующего класса, в функции вместо element пишем elements
#на выходе у нас будет список всех соответствующих элементов на странице

title = browser.find_elements(by=By.CLASS_NAME, value='projects__title')
competition = browser.find_elements(by=By.CLASS_NAME, value='contest')
price = browser.find_elements(by=By.CLASS_NAME, value='projects__price')
direction = browser.find_elements(by=By.CLASS_NAME, value='direction')
descr = browser.find_elements(by=By.CLASS_NAME, value='projects__descr')

In [ ]:
title[10].text

'Футбол - играет вся семья'

In [ ]:
description = descr[1].text
description

'Санкт-Петербург\nпобедитель конкурса\n69,00\nЗаявка 22-1-015195 от 15.10.2021'

Чтобы вытащить из этой строки нужные нам значения, мы будем использовать **регулярные выражения**. При работе с неструктурированными и необработанными данными это - очень полезная вещь. Советую изучить их самостоятельно - пригодится)

In [ ]:
import re #модуль для работы с регулярными выражениями

In [ ]:
#регулярные выражения для парсинга нашего описания

region = re.search(r'^[А-ЯЁ]\w+\W*\w+', description).group(0)
status = re.search(r'\n[а-я-ё]\w+\W*[а-яё]\w+\n', description).group(0).replace('\n', '')
rate = re.search(r'\n\d+\,*\d*\n', description).group(0).replace('\n', '')
app_id = re.search(r'\s\d+\-*\d*\-*\d+\s', description).group(0).replace('\s', '')
app_date = re.search(r'\d\d\.\d\d\.\d{4}', description).group(0).replace('\s', '')

In [ ]:
app_date

'15.10.2021'

In [ ]:
#теперь ОБЯЗАТЕЛЬНО нужно запустить эту команду, чтобы закрыть браузер

browser.quit()

In [ ]:
#чтобы пройти по всем страницам и собрать нужные данные мы создадим цикл (для всех страниц, кроме первой)
for i in range(1, 109):
    url = f'https://xn--80afcdbalict6afooklqi5o.xn--p1ai/public/application/cards?SearchString=&Statuses%5B0%5D.Name=%D0%BF%D0%BE%D0%B1%D0%B5%D0%B4%D0%B8%D1%82%D0%B5%D0%BB%D1%8C+%D0%BA%D0%BE%D0%BD%D0%BA%D1%83%D1%80%D1%81%D0%B0&Statuses%5B1%5D.Name=%D0%BD%D0%B0+%D0%BD%D0%B5%D0%B7%D0%B0%D0%B2%D0%B8%D1%81%D0%B8%D0%BC%D0%BE%D0%B9+%D1%8D%D0%BA%D1%81%D0%BF%D0%B5%D1%80%D1%82%D0%B8%D0%B7%D0%B5&Statuses%5B2%5D.Name=%D0%BF%D1%80%D0%BE%D0%B5%D0%BA%D1%82+%D0%BD%D0%B5+%D0%BF%D0%BE%D0%BB%D1%83%D1%87%D0%B8%D0%BB+%D0%BF%D0%BE%D0%B4%D0%B4%D0%B5%D1%80%D0%B6%D0%BA%D1%83&RegionId=&ContestDirectionTenantId=&CompetitionId=91c698a4-c11d-4ad6-9148-03d58cd7c837&DateFrom=&DateTo=&Statuses%5B0%5D.Selected=false&Statuses%5B1%5D.Selected=false&Statuses%5B2%5D.Selected=false&page={str(i)}'
    browser = webdriver.Chrome(ChromeDriverManager().install())
    browser.set_window_size(1280, 1024)
    browser.get(url)
    #здесь прописываем все элементы, которые хотим добавлять
    #стандартная уже для вас процедура - сохранение элементов в списки для создания таблицы

    browser.quit() #закрываем браузер        

## Домашнее задание ##

Используя код из Примера №2 из этого блокнота, выгрузите данные о **победителях второго конкурса 2021 года** в таблицу csv.

**Итоговая таблица должна содержать следующие поля:**

- Название проекта;
- Сумма гранта;
- Направление гранта;
- Регион участника.

Формат сдачи ДЗ: файл с кодом (jpynb или py), таблица csv.